In [1]:
using DataFrames, ForwardDiff

In [2]:
# Basic trust region with conjugate gradient, logit.

df = readtable("../data/parsed_model_australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b

25

In [3]:
immutable BasicTrustRegion{T<:Real}
    η1::T
    η2::T
    γ1::T
    γ2::T
end

function BTRDefaults()
    return BasicTrustRegion(0.01, 0.9, 0.5, 0.5)
end

type BTRState
    iter::Int64
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64
    tol::Float64

    function BTRState()
        state = new()
        state.tol = 1e-6
        return state
    end
end

In [4]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η1
        return true
    else
        return false
    end
end

function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η2
        stepnorm = norm(state.step)
        state.Δ = min(1e20, max(4*stepnorm, state.Δ))
    elseif state.ρ >= b.η1
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

In [5]:
function ConjugateGradient(A::Matrix, b::Vector, x0::Vector, tol::Float64 = 1e-6)
    n = length(x0)
    x = x0
    g = b+A*x
    d = -g
    k = 0
    tol2 = tol*tol
    while dot(g, g) > tol2
        Ad = A*d
        normd = dot(d, Ad)
        α = -dot(d, g)/normd
        x += α*d
        g = b+A*x
        γ = dot(g, Ad)/normd
        d = -g+γ*d
        k += 1
    end
    normd = dot(d, A*d)
    α = -dot(d, g)/normd
    x += α*d
    return x
end

ConjugateGradient (generic function with 2 methods)

In [6]:
function btr(f::Function, g!::Function, H!::Function, Step::Function,
        x0::Vector, state::BTRState = BTRState(), ApproxH::Bool = false)
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n = length(x0)
    tol2 = state.tol*state.tol
    state.g = zeros(n)
    H = eye(n, n)
    fx = f(x0)
    g!(x0, state.g)
    state.Δ = 0.1*norm(state.g)
    if ApproxH
        y = zeros(n)
        gcand = zeros(n)
    else
        H!(x0, H)
    end
    nmax = 500

    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while dot(state.g, state.g) > tol2 && state.iter < nmax
        state.step = Step(H, state.g, state.x)
        state.xcand = state.x+state.step
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))
        if ApproxH
            g!(state.xcand, gcand)
            y = gcand-state.g
            H = H!(H, y, state.step)
        end
        if acceptCandidate!(state, b)
            state.x = copy(state.xcand)
            if !ApproxH
                g!(state.x, state.g)
                H!(state.x, H)
            else
                state.g = copy(gcand)
            end
            fx = fcand
        end
        updateRadius!(state, b)
        state.iter += 1
    end
    return state.x, state.iter
end

btr (generic function with 3 methods)

In [7]:
function individual(β::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    data = convert(Array, df[i, 2:n_parameters])
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function logit(β::Vector)
        t = 0.0
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return logit
end

individual (generic function with 1 method)

In [8]:
function f(β::Vector)
    model = 0.0
    i = 1
    while i <= n_individuals
        logit = individual(β, i)
        model += log(logit(β))
        i += 1
    end
    return -model/n_individuals
end

f (generic function with 1 method)

In [9]:
function g(x::Vector)
    t = zeros(length(x))
    for i = 1:n_individuals
        logit = individual(x, i)
        t += (1/logit(x))*ForwardDiff.gradient(logit, x)
    end
    return -t/n_individuals
end

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [10]:
function H(x::Vector)
    return ForwardDiff.hessian(f, x)
end

function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [11]:
function BHHH(x::Vector)
    B = zeros(length(x), length(x))
    for i = 1:n_individuals
        logit = individual(x, i)
        g = ForwardDiff.gradient(logit, x)
        l = (1/logit(x))
        B += (g/l)*(g'/l)
    end
    return B
end

function BHHH!(x::Vector, storage::Matrix)
    s = BHHH(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [12]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    Bs = B*s
    B[1:n, 1:m] = B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

BFGS! (generic function with 1 method)

In [13]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    Bs = B*s
    B[1:n, 1:m] = B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

SR1! (generic function with 1 method)

In [14]:
btr(f, g!, H!, ConjugateGradient, zeros(6), BTRState())

([5.20743, 3.86904, 3.16319, 0.0132871, -0.0961247, -0.0155015], 5)

In [15]:
btr(f, g!, BHHH!, ConjugateGradient, zeros(6), BTRState())

([5.20743, 3.86904, 3.16319, 0.0132871, -0.0961247, -0.0155015], 500)

In [16]:
btr(f, g!, BFGS!, ConjugateGradient, zeros(6), BTRState(), true)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 500)

In [17]:
btr(f, g!, SR1!, ConjugateGradient, zeros(6), BTRState(), true)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 500)